<b>20.Valid Parentheses </b>. 

using stack. loop the array, if it is the start sign, put it in stack. if it is an end side, pop the stack to get the last item, compare with the correpsonding item's start sign. If not match, return False direclty, else contingue.  
at the end, if the whole stack is empty, then True

In [4]:
def isValid( s):
    """complexity O(N), search in keys is O(1), pop for list is O(1) too"""
    mapping = {')': '(', '}': '{', ']': '['}
    n = len(s)
    if n%2==1:
        return False
    stack = []
    for item in s:
        # start of the parentheses
        if item not in mapping.keys(): # adding start sign
            stack.append(item)
        else:
            if (not stack) or (stack.pop()!=mapping[item]):
                return False
    return len(stack)==0  # in case any start sign does not have matching ending sign

In [2]:
s="()[]{}"
isValid( s)

True

In [3]:
s="(("
isValid( s)

False

<b> 1249 Minimum Remove to Make Valid Parentheses </b>

using stack to identify which ( does not have matching ending ). using remove set to find ending parenthese ) that does not have matching openning (

In [1]:
def minRemoveToMakeValid(s: str):
    """O(N)"""
    remove = set()
    stack=[]
    for i, c in enumerate(s):
        if c=='(':
            stack.append(i)
        elif c==')':
            if not stack:
                remove.add(i) # a ) with no matching (
            else:
                stack.pop()
    remove = remove | set(stack) # stack may have ( that does not have a matchig )
    result = []
    for i, c in enumerate(s):
        if i not in remove:
            result.append(c)
    return "".join(result)

In [2]:
s = "lee(t(c)o)de)"
minRemoveToMakeValid(s)

'lee(t(c)o)de'

<b> 224 Basic Calculator </b>

also about parenthsis. Here assume the expression string is valid.  
using stack to save value and sign for a new start, which indicates by (
need to consider the case that number string like "23", number can be >9

"(" means a new start, push to stack, ")" means the en d of a inner (), pop value from stack to resume calculation of outer ()

In [3]:
def calculate(s):
    """stack
    need to consider case like "45+23", number can >9
    """
    sign = 1
    result = 0 # store current()'s result, also works for expression without any ()
    i = 0
    stack = []
    while i < len(s):
        if s[i].isnumeric():
            number = int(s[i])
            while i+1 < len(s) and s[i+1].isnumeric():
                number = number*10 + int(s[i+1])
                i+=1
            result += number*sign
        elif s[i]=="+":
            sign = 1
        elif s[i]=="-":
            sign = -1
        elif s[i] == "(":
            # a new start
            stack.append(result)
            stack.append(sign)
            result = 0
            sign = 1
        elif s[i] == ')':
            # the number in inner () completes its calculation, now can calculate it with number in the queue for outer ()
            # first pop is the sign, second pop is the number
            result = result*stack.pop() + stack.pop()
        i += 1
    return result


In [4]:
s="34 + 3"
calculate(s)

37

<b> 227 basic calculator II </b>

<b>way 1: my way  </b>
flatten the operation * \ to lower level operation + -  
then go over the left over stack, just adds all the number up  
<b>key part</b> is given the first number a positive sign (the question assumes no negative value in the expresssion)  

In [5]:
def calculate(s):
    """
    no parenthese, but the sign has different priority level
    using stack maybe with recursion
    """
    num_stack = [] # for case like "3", still work
    sign_stack = [1] # level one sign: +:1, -:-1
    i = 0
    num = 0
    def helper(i, num):
        # helper func to get number after * / sign
        while i+1 < len(s) and s[i+1]==" ":
            i += 1
        while i+1 < len(s) and s[i+1].isnumeric():
            num = num*10 + int(s[i+1])
            i += 1
        return i, num
    # flattern priority from */ to all + -
    while i < len(s):
        if s[i].isnumeric():
            num = int(s[i])
            while i+1 < len(s) and s[i+1].isnumeric():
                num = num*10 + int(s[i+1])
                i += 1
            num_stack.append(num)
            num = 0
        elif s[i] == "+":
            sign_stack.append(1)
        elif s[i] == '-':
            sign_stack.append(-1)
        elif s[i] == '*':
            i, num = helper(i, num)
            num_stack.append(num * num_stack.pop())
            num = 0
        elif s[i] == '/':
            i, num = helper(i, num)
            num_stack.append(num_stack.pop()//num)
            num = 0
        i += 1
    result = num_stack.pop() * sign_stack.pop()
    while num_stack:
        # assume the expression is always valid, so sign stack always has corresponding pair
        result += sign_stack.pop()*num_stack.pop()
    return result

In [6]:
s = "3+2*2"
calculate(s)

7

<b> way 2</b> => looks more concise. 

also key part is to give the first number a positive sign, but no need to keep check of the stack of sign, only current value. when encounter a sign, push the value before this sign together with the sign before this value: a,b,c => 1+2 => +1+2, when at second +, calculate 1 and the + infront 1 then push the result into a stack.

the parallel if statement helps in edge and special cases like " " char and at the end of string


In [7]:
def calculate(s):
    """
    using a pre sign val, like a positive number its sign by default is +, here we explicitly assign the + to it.
    everytime when you encouter a sign, first push the previous number with its previous sign, then update the current sign.
    the two parellel if check can eliminate edge cases, like at empty space and at the end of string.
    """

    i = 0
    sign = "+" # initial sign
    stack = []
    num = 0
    for i in range(len(s)):
        if s[i].isnumeric():
            num = num*10 + int(s[i])
        # the parallel if statement works even when i is at the end of str
        if not s[i].isnumeric() and s[i]!=" " or i==len(s)-1:
            # not a number or a space, then must be a sign
            # check the previous sign first, then update the sign value
            if sign == '-':
                stack.append(-1*num)
            elif sign == '+':
                stack.append(num)
            elif sign == '*':
                stack.append(stack.pop()*num)
            elif sign == '/':
                # NOT work => -3//2 = -2 isntead of -1, so "14-3/2" wont return 13 but 12 
                val = stack.pop()
                if val < 0:
                    stack.append((-val//num)*-1)
                else:
                    stack.append(val//num)
            num = 0
            sign = s[i]

    # now stack has values already cleaned with sign
    result = stack.pop()
    while stack:
        result += stack.pop()
    return result

<b> 301 remove invalid parentheses </b>  
step 1: remove extra )  
step 2: remove extra (  
two steps can use same code by reversing the order of checking parenthese

how to determine if there is an extra ( or )? check and count along the string, whenever you have more ) than (, then you encouter an extra ) and remove the first encoutering one can solve edge case like )(), also avoid duplicates.

keep check of the last visited position as the next starting position (wont duplicately check that position because old string removed a char to becomes the new string for the next check, due to the removal, the last visited position shifted left for one position, and i happens to point to the next valid position for checking.

same idea for the last removal position becomes the next valid removal starting position due to old s†ring shedded for one char.

the return sta†ement is smart. if you enter the recursive function call, that means you havent fully finished the checking of extra ( sign, so cant proceed to †he check of extra ).


In [8]:
def removeInvalidParentheses(s):
    def remove_helper(s, res, start, last_r, mapping):
        """
        start: start index where to check the invalid parenthsis
        last_r: index of last removal, now becomes the start of next removable position
        """
        count = 0
        for i in range(start, len(s)):
            if s[i]==mapping[0]: count += 1
            if s[i]==mapping[1]: count -= 1
            if count >=0: continue # still have '(' left unpaired
            # otherwise an extra ')' need to be removed, from position last_r to i, last_r is the position where last item is removed, now become the next removal position (valid because of the char shifting)
            for j in range(last_r, i+1):
                # remove the first ')' in this range, to avoid duplicats
                # ex: ()()) => (()) or ()()
                if s[j]==mapping[1] and (j==last_r or s[j-1]!=mapping[1]):
                    # s[j-1]!=mapping[1] can help ensure ()())=>()() wont produce duplciates, so only remove the ) at position s[3] not s[4]
                    # why i and j works?
                    # because remove one char, now i points to the next starting point of s that can start scan for unncessary ), and j points to the next removal point of s.
                    remove_helper(s[:j]+s[j+1:], res,i, j, mapping)
            # return is smart! 
            # if ever enter the consecutive remove_helper call, current string is not ready from left to right yet (steps to check extra ')'' ), so cant proceed to check from right to left => the reverse order to check extra '(''
            return
        # start the following piece only after one direction is clean
        reversed_s = s[::-1]
        if mapping[0] == '(': # it previously remove from left to right, find (, now can remove from right to left
            remove_helper(reversed_s, res, 0, 0, [')', '('])
        else: # already removed from left to right and right to left, so ready
            res.append(reversed_s)


    res = []
    # mapping = ['(',')']
    remove_helper(s, res, 0, 0, ['(',')'])
    return res



In [9]:
s= "(a)())()"
removeInvalidParentheses(s)

['(a())()', '(a)()()']

### Monotone Stack

a data structure that keeps its element either entirely in non-increasing or non-decreasing order => a DP technique

time complexity if Linear

<b> 84 Largest Rectangle in Histogram </b>

monotone stack save the index, the corresponding element is in <b>increasing</b> order (if encouter a decreasing or equal one, pop stack).   
index is used to find the width of valid rectangle, the actual value of the heights array is the height of the rectangle.

case 1: x1 is current i, x0 is previous smaller item's index
case 2: x1 is n, the length, x0 is the previous smaller item's index
height is the item poping out from stack


In [22]:
def largestRectangleArea(heights):
    stack=[-1]  # store the index, using -1 to store the start of the array, to save the max possible width
    max_area = 0
    n = len(heights)
    for i in range(len(heights)):
        # if the stack is not empty and the last item in stack is greater than current item in height
        while stack[-1]!=-1 and heights[stack[-1]]>=heights[i]:
            h = heights[stack.pop()]
            w = i-stack[-1]-1 # important step, cant using i-the_prev_stack_pop_value, because it will elimite where the array starts, for example [5,4,1,2], for 4, if using i-4_index, w = 1, but actually 4's w should be 2, from start of array to where 4 stands 
            max_area = max(max_area, h*w)
        stack.append(i)
    # now the stack is non-decreasing, pay special attention to include where the stack starts
    while stack[-1]!=-1:
        # key is n-stack[-1]-1 to get the width
        # [2,1,4,5,2,3], when it has [1,2,3] left, the width for 2 is actually 4, not 2 (if using n-indx_of_2)
        max_area = max(max_area, heights[stack.pop()]*(n-stack[-1]-1))
    return max_area

In [23]:
heights = [2,1,5,6,2,3]
largestRectangleArea(heights)

10

there are other easier understanding ways, but <b>too slow</b> to pass all the tests, especially when the array is sorted

In [5]:
def largestRectangleArea(heights):
    """divide and concur, make the problem into smaller problem
        still too slow, avg case O(NlogN), worse case is O(N^2) if array is sorted
    """
    max_area = 0
    def helper(start, end):
        nonlocal max_area
        if start > end:
            return
        min_index = start
        for i in range(start, end+1):
            if heights[i]<heights[min_index]:
                min_index = i 
        max_area = max(max_area, heights[min_index]*(end-start+1))
        helper(start, min_index-1)
        helper(min_index+1, end)

    helper(0, len(heights)-1)
    return max_area

In [6]:
heights = [2,1,5,6,2,3]
largestRectangleArea(heights)

10

<b> 503 Next Greater Element II </b>

using monotone stack, and using 2*len(array) and % to construct the circle.  
here the monotone stack stores <b>non-increasing</b> value  
either the stack is empty, meaning all values find its next larger elements (which should not be true because the largest item will have -1)
or loop over the circle completely, marking the end of the finding.

In [7]:
def nextGreaterElements(nums):
    n = len(nums)
    stack = []  # save index, not value
    result = [-1]*n

    # 2*n to create the circle, i%n is the real index
    for i in range(2*n):
        val = nums[i%n]
        while stack and nums[stack[-1]]<val:
            # find the next larger number for nums[stack[-1]]
            result[stack.pop()] = val
        # only append when i is valid, to avoid duplicates
        if i < n:
            stack.append(i)
    return result


In [8]:
nums = [1,2,1]
nextGreaterElements(nums)

[2, -1, 2]

<b> 907 sum of subarray minimums </b>

Monotone again! find each element's previous less num's index, and next less num's index, using the index to calculate there is how many subarray that has this element as the minimum value. 

the first stack is a non-decreasing stack, p_value <= c_value. it finds the previous smaller or equal value, 
the second stack is a non-decreasing stack, p_value <= c_value. if finds the next stricktly smaller value

the other way around is working too, see second part of code

In [9]:
def sumSubarrayMins(A):
    n = len(A)
    mod = 10**9+7
    prev_less, next_less = [-1]*n, [n]*n
    
    # find prev less num's index
    stack = [] # save index, temp
    for i in range(n):
        # if A[stack[-1]]>=A[i] here, then next_less part should be strict condition, to avoid duplicates
        while stack and A[stack[-1]] > A[i]:
            # stack stores all the previous value
            stack.pop() # pop all previous items that is > A[i]
        if stack:
            # not pop(), like [3,1,5,4] for 4 need to find 1, if pop, cant find 1 anymore
            # find current item's prev less!
            prev_less[i] = stack[-1]  # item i's previous less is current top of stack
        stack.append(i)
    
    # find next_less num's index    
    stack = []  # temp again
    for i in range(n):
        while stack and A[stack[-1]] > A[i]:
            # find next less index for the items in the stack! 
            next_less[stack.pop()] = i
        stack.append(i)
    ans = 0
    for i in range(n):
        # this many different subarray: (i-prev_less[i])*(next_less[i]-i)  that has A[i] as its min value
        ans += A[i]*(i-prev_less[i])*(next_less[i]-i)
    return ans%mod

In [11]:
A = [3,1,4,1]
sumSubarrayMins(A)

15

the above using A[stack[-1]] > A[i], so for example [3,1,4,1], the prev_less is [-1,-1,1,1], (not strict for prev_less, see last 1). next_less is [1,4,3,4] (see all 1 has no next less value).  
same if i use A[stack[-1]] >= A[i] for both while loop, for example [3,1,4,1], the prev_less would be [-1,-1,1,-1] (both 1 has no prev less value), next_less would be [1,3,3,4] (the first 1 has next less value)


In [12]:
def sumSubarrayMins(A):
    n = len(A)
    mod = 10**9+7
    prev_less, next_less = [-1]*n, [n]*n
    
    # find prev less num's index
    stack = [] # save index, temp
    for i in range(n):
        # if A[stack[-1]]>=A[i] here, then next_less part should be strict condition, to avoid duplicates
        while stack and A[stack[-1]] >= A[i]:
            # stack stores all the previous value
            stack.pop() # pop all previous items that is > A[i]
        if stack:
            # not pop(), like [3,1,5,4] for 4 need to find 1, if pop, cant find 1 anymore
            # find current item's prev less!
            prev_less[i] = stack[-1]  # item i's previous less is current top of stack
        stack.append(i)
    
    # find next_less num's index    
    stack = []  # temp again
    for i in range(n):
        while stack and A[stack[-1]] >= A[i]:
            # find next less index for the items in the stack! 
            next_less[stack.pop()] = i
        stack.append(i)
    ans = 0
    for i in range(n):
        # this many different subarray: (i-prev_less[i])*(next_less[i]-i)  that has A[i] as its min value
        ans += A[i]*(i-prev_less[i])*(next_less[i]-i)
    return ans%mod

In [13]:
A = [3,1,4,1]
sumSubarrayMins(A)

15

Idea is using a monotonous non-descreasing stack to store the left boundary and right boundary where the number (each number) is the minimal number of the boundary range. Using the boundary index to calculate there are how may subarray that has this number as min value  
So same idea as way 1, but more concise implementation  
Adding 2 infinite value at the head and tail just to make sure the index 0 and len(array) are used  
For example array [3,1,4,1]. => [ -inf, 3, 1, 4, 1, -inf ]  
For 3: [ -inf, 3, 1].   3*( 1-0)*(2-1)  
For 1: [inf, 1, inf].  1* (2-0) * (5-2)  
For 4: [1, 4, 1]. 4* ( 3-2) *(4-3)  
For 1: [1, 1, -inf ] 1*(4-2) *(5-4).   
=> total 3+6+4+2 = 15

In [16]:
def sumSubarrayMins(A):
    res = 0
    stack = [] # store index, the stack points to value non-descreasing
    A = [float('-inf')] + A + [float('-inf')]
    for i, n in enumerate(A):
        while stack and A[stack[-1]] > n:
            cur = stack.pop()
            # stack[-1], curr, i
            # stack[-1]: left boundary (include, stack[-1]<=cur value)
            # i: right boundary (exclude, curr value > n)
            res += A[cur] * (cur - stack[-1])*(i-cur)
        stack.append(i)
    return res%(10**9+7)

In [15]:
A = [3,1,4,1]
sumSubarrayMins(A)

15

<b> 901 online stock span </b>

add a self.index to for better reference, treat the input as array  
using index to calcuate the distance. self.index is the index of current item. 
find the previous larger item's index as left boundary.

In [17]:
class StockSpanner:

    def __init__(self):
        self.stack = [(0, float('inf'))]  # (index, value)
        self.index = 0
        

    def next(self, price: int) -> int:
        self.index+=1
        # <= can deal with duplicated value
        while self.stack and self.stack[-1][1] <= price:
            self.stack.pop()
        # previous large item's index, as left boundary
        prev_large = self.stack[-1][0]
        self.stack.append((self.index, price))
        return self.index - prev_large

In [18]:
obj = StockSpanner()
print(obj.next(100))
print(obj.next(80))
print(obj.next(60))
print(obj.next(70))
print(obj.next(60))
print(obj.next(75))
print(obj.next(85))
print(obj.next(85))

1
1
1
2
1
4
6
7


<b> 739 Daily Temperature </b>  

When array[ stack[-1] ] < current value, then the value at stack[-1] find its next larger value

NOTE: dont compare stack[-1] with v

def dailyTemperatures( T) :
    n = len(T)
    stack = [] # save index
    res = [0]*n

    # find next larger value, get the distance
    for i, v in enumerate(T):
        # remember compare the actual value in T with V
        while stack and T[stack[-1]]<v:
            ind = stack.pop()
            res[ind] = i-ind
        stack.append(i)
    return res

In [21]:
T = [73, 74, 75, 71, 69, 72, 76, 73]
dailyTemperatures(T)

[1, 1, 4, 2, 1, 1, 0, 0]

<b> 85 Maximal Rectangle </b>

way 1: using the function defined in 84 Largest Rectangle in Histogram as helpfer function.  
consider each row in the matrix as the new base of the histogram    
for example:  
[
  ["1","0","1","0","0"],  
  ["1","0","1","1","1"],  
  ["1","1","1","1","1"],  
  ["1","0","0","1","0"]  
]  

row 0 as base: height the histogram is [1, 0, 1, 0, 0]  
row 1 as base: height the histogram is [2, 0, 2, 1, 1]  
row 2 as base: height the histogram is [3, 1, 3, 2, 2]  
row 4 as base: height the histogram is [4, 0, 0, 3, 0]  
loop each row, using the helper function to calcuate the max rectangle for each histogram (row). get the max value

In [24]:
def maximalRectangle(matrix):
    """using dynamic programming and monontone stack"""
    def helper_get_max_hist(heights):
        stack = [-1] # save index, used to calculate width
        max_area = 0
        for i in range(len(heights)):
            while stack[-1]!=-1 and heights[stack[-1]]>=heights[i]:
                h = heights[stack.pop()]
                w = i - stack[-1]-1
                max_area = max(max_area, h*w)
            stack.append(i)
        while stack[-1]!=-1:
            h = heights[stack.pop()]
            w = len(heights) - stack[-1]-1
            max_area = max(max_area, h*w)
        return max_area

    if not matrix:
        return 0
    max_area = 0
    dp = [0] * len(matrix[0]) # initiate a space for each column => height
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            # update the size of this row's histogram by using last row's histogram size
            dp[j] = dp[j]+1 if matrix[i][j]=="1" else 0
        # consider each row as the base of the histogram, get the max area of the sub histogram
        # dp is the array representing size of each bar in a histogram
        max_area = max(max_area, helper_get_max_hist(dp))

    return max_area

In [25]:
matrix = [["1","0","1","0","0"],
  ["1","0","1","1","1"],
  ["1","1","1","1","1"],
  ["1","0","0","1","0"]]
maximalRectangle(matrix)

6

way 2, similar idea, loop each row, using each row as base, for each point at the row, at that position, get its left boundary, right boundary, height boundary, to calculate the area.

In [26]:
def maximalRectangle(matrix) -> int:
    """dynamic programming"""
    if not matrix: return 0
    m = len(matrix)
    n = len(matrix[0])
    left = [0]*n  # n columns, initial value:index 0?
    right = [n]*n  # n columns, initial value:index n => out of boundry on purpose
    height = [0]*n # n columns
    max_area = 0

    for i in range(m):
        cur_left, cur_right = 0, n  # initial value is index
        # cur_left: most recent index position from left where value is 1
        # cur_right: most recent index position from right where value is 1
        
        # update height
        for j in range(n):
            if matrix[i][j]=='1':
                height[j]+=1
            else:
                height[j] = 0
        # update left
        for j in range(n):
            if matrix[i][j]=='1':
                left[j] = max(left[j], cur_left)
            else:
                left[j]=0  # back to initial value
                cur_left = j+1
        # update right
        for j in range(n-1, -1, -1):  # loop from right to left
            if matrix[i][j]=='1':
                right[j]=min(right[j], cur_right)
            else:
                right[j]=n # back to initial value
                cur_right = j
        for j in range(n):
            max_area = max(max_area, height[j]*(right[j]-left[j]))

    return max_area

In [27]:
matrix = [["1","0","1","0","0"],
  ["1","0","1","1","1"],
  ["1","1","1","1","1"],
  ["1","0","0","1","0"]]
maximalRectangle(matrix)

6

<b>1081 smallest subsequence of distinct characters </b>

Find the index of last occurrence for each character.
Use a stack to keep the characters for result.
Loop on each character in the input string S,
if the current character is smaller than the last character in the stack,
and the last character exists in the following stream,
we can pop the last character to get a smaller result.

In [30]:
def smallestSubsequence(text):
    ind_dict = {c: i for i, c in enumerate(text)}
    stack = [] # store actual character
    for i, c in enumerate(text):
        if c in stack: continue  # important step to avoid duplicates
        # last char in stack is larger than current one, and the last char exist in later position, so feel free to pop it out
        while stack and stack[-1]>c and ind_dict[stack[-1]]>i:
            stack.pop()
        stack.append(c)
    return "".join(stack)
                

In [29]:
text = "cdadabcc"
smallestSubsequence(text)

'adbc'

In [32]:
text="cbacdcbc"
smallestSubsequence(text)

'acdb'

<b> 402 remove k digits </b>

how to determine which k digits to remove to make the left number the smallest.   
monotone stack, saving the non-desreasing digits in the limited length, which will make the left number the smallest.
after constructing the non-descreasing stack, it can happen that k is not used up yet. now the largest digits are all at the tail, so remove from the tail.  
remember to remove leading zeros in the result

In [33]:
def removeKdigits(num, k):
    if k == len(num):
        return "0"
    stack =[]
    for n in num:
        # if stack[-1]>=n, wont get the "112" k=1 correctly
        while k and stack and stack[-1]>n: 
            k -=1
            stack.pop()
        stack.append(n)
    # like stack is ascending already, but k still not used up yet
    if k:
        stack = stack[:-k]
    # prevent leading zero
    return "".join(stack).lstrip("0") or "0"

In [35]:
num = "112"
k = 1
removeKdigits(num, k)

'11'

<b> 132 pattern </b>

for example [-1,3,2,0], an valid 132 could be [-1,2,0], that means the sequence no need to be subarray

the idea here is the loop from the end of array.
the value popped from stack is s3, the value that is larger then s3 to trigger it popping out is s2, now if there exist a s1 < s3, we can return True

here s1 < s3 < s2

In [36]:
def find132pattern(nums):
    s3 = float('-inf')
    stack = []  # save actual value
    for i in range(len(nums)-1, -1, -1):
        if nums[i] < s3: return True
        # stack is non-increasing
        while stack and stack[-1] < nums[i]:
            # maximum candidate for s3 is always the recently popped number from the stack
            s3 = stack.pop()
            # nums[i] is the s2, so we got s2 and s3, next is to see if there is s1
        stack.append(nums[i])
    return False

In [37]:
nums= [-1,3,2,0]
find132pattern(nums)

True

<b> 239 sliding window maximum </b>

monotone stack but using deque, smart to check index in the queue against the i-k value, to see if the index in queue is within valid window size or not

using a deque as monotone stack to save possible max value,  
should pop out the left most item in the stack then the window sliding over it already, it is no used anymore.

the first element in the stack is always the max value in the window

In [38]:
from collections import deque
def maxSlidingWindow(nums, k):
    d = deque()  # save index
    result = []
    for i, n in enumerate(nums):
        while d and nums[d[-1]]<n:
            # smaller number from previous position wont be the max number in the window where n exist, cz at least n will be the one
            d.pop()
        d.append(i)
        # for every i, check if current deque's elements is in the k window or not
        if d[0] == i-k:
            # d[0] is not in the current k window
            # like 2,3,4,5, k=3, i=5, 2 should not be included
            d.popleft()
        # first 0 - k-2 element should not be included
        if i>=k-1:
            result.append(nums[d[0]])
    return result

In [39]:
nums = [1,1,1,1,1,1]
k = 3
maxSlidingWindow(nums, k)

[1, 1, 1, 1]

In [40]:
nums = [1,2]
k = 3
maxSlidingWindow(nums, k)

[]